In [1]:
import pandas as pd

df = pd.read_csv("./position_wt_proc.csv")

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import numpy as np

In [ ]:
class PositionalEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Input size updated to 10: original 2 + (sin, cos values for orders 1 to 4) * 2
        self.fc1 = nn.Linear(18, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 7)
        self.ac1 = nn.LeakyReLU()
        self.norm1 = nn.BatchNorm1d(256)
        self.norm2 = nn.BatchNorm1d(256)

    def forward(self, x):
        x[:, 0] *= torch.pi / 180
        x[:, 1] *= torch.pi / 360
        # Compute sine and cosine for orders 1 to 4
        features = [x]  # Start with the original input
        for order in range(1, 10):
            features.append(torch.sin(order * x))  # sin(nx)
            features.append(torch.cos(order * x))  # cos(nx)

        for order1 in range(1, 10):  # second order
            for order2 in range(1, 10):
                features.append(torch.sin(order1 * x) * torch.sin(order2 * x))
                features.append(torch.sin(order1 * x) * torch.cos(order2 * x))
                features.append(torch.cos(order1 * x) * torch.cos(order2 * x))

        # Concatenate all features along the last dimension
        x = torch.cat(features, dim=1)

        # Forward pass through the network
        x = self.norm1(self.ac1(self.fc1(x)))
        x = self.norm2(self.ac1(self.fc2(x)))
        x = self.fc3(x)
        return x, F.softmax(x, dim=1)[:, :6]

In [4]:
class PositionalDataset(Dataset):
    def __init__(
        self, df, nstd=1 / 2.6, device="mps"
    ):  # 2.6: 99%, 3.3: 99.9%, 3.9: 99.99%, 4.5: 99.999%
        self.df = df
        self.nstd = nstd
        self.device = device

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        lat = self.df["lat"].iloc[idx]
        lon = self.df["lon"].iloc[idx]
        dlat = self.df["dlat"].iloc[idx]
        dlon = self.df["dlon"].iloc[idx]
        return torch.normal(
            torch.tensor((lat, lon), dtype=torch.float32),
            self.nstd * torch.tensor((dlat, dlon), dtype=torch.float32),
        ).to(self.device), torch.tensor(
            self.df[["wmgo", "wal2o3", "wsio2", "wcao", "wtio2", "wfeo"]]
            .iloc[idx]
            .values,
            dtype=torch.float32,
        ).to(
            self.device
        )


train_dl = DataLoader(PositionalDataset(df), batch_size=1024, shuffle=True)

In [5]:
# k means cross validation
k = 10
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
dl = len(shuffled_df) // k

test_dfs = [shuffled_df[i * dl : (i + 1) * dl] for i in range(k - 1)]
test_dfs.append(shuffled_df[(k - 1) * dl :])


train_dfs = [shuffled_df.drop(test_df.index) for test_df in test_dfs]
train_dls = [
    DataLoader(PositionalDataset(train_df), batch_size=1024, shuffle=True)
    for train_df in train_dfs
]
test_dls = [
    DataLoader(PositionalDataset(test_df), batch_size=1024, shuffle=True)
    for test_df in test_dfs
]

models = [PositionalEncoder().to("mps") for _ in range(k)]
optimizers = [optim.Adam(model.parameters(), lr=1e-3) for model in models]

In [6]:
epochs = 1000
mse = [[[] for _ in range(k)] for _ in range(epochs)]
msev = [[[] for _ in range(k)] for _ in range(epochs)]
cnts = [[0 for _ in range(k)] for _ in range(epochs)]
cntsv = [[0 for _ in range(k)] for _ in range(epochs)]
for epoch in range(epochs):
    print("epoch", epoch)
    for i in range(k):
        # print("model", i)

        model = models[i]
        optimizer = optimizers[i]
        model.train()
        for input, target in train_dls[i]:
            _, output = model(input)
            mse_loss = F.mse_loss(output, target)
            # kld_loss = F.kl_div(output, target, reduction="batchmean")
            loss = mse_loss  # 0.9 * mse_loss + 0.1 * kld_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            cnts[epoch][i] += input.shape[0]
            mse[epoch][i].append(mse_loss.item() * input.shape[0])
        # print("Training MSE:", sum(mse[epoch][i]) / cnts[epoch][i])
        model.eval()
        for input, target in test_dls[i]:
            _, output = model(input)
            mse_loss = F.mse_loss(output, target)
            msev[epoch][i].append(mse_loss.item() * input.shape[0])
            cntsv[epoch][i] += input.shape[0]
        # print("Testing MSE:", sum(msev[epoch][i]) / cntsv[epoch][i])

    print(
        "Average Training MSE:",
        sum([sum(mse[epoch][i]) / cnts[epoch][i] for i in range(k)]) / k,
    )
    print(
        "Average Testing MSE:",
        sum([sum(msev[epoch][i]) / cntsv[epoch][i] for i in range(k)]) / k,
    )

    print(
        "Statistical Error +/-",
        np.sqrt(sum([sum(msev[epoch][i]) / cntsv[epoch][i] for i in range(k)])) / k,
    )

epoch 0
Average Training MSE: 0.019289671020231464
Average Testing MSE: 0.019864177938676543
Statistical Error +/- 0.04456924717636203
epoch 1
Average Training MSE: 0.012882419571815625
Average Testing MSE: 0.01463449744901187
Statistical Error +/- 0.03825506168994094
epoch 2
Average Training MSE: 0.010728235605148502
Average Testing MSE: 0.010707568568577422
Statistical Error +/- 0.03272242131716023
epoch 3
Average Training MSE: 0.009371339189353043
Average Testing MSE: 0.008616782433456562
Statistical Error +/- 0.029354356462808994
epoch 4
Average Training MSE: 0.008367141930648744
Average Testing MSE: 0.007643514566004386
Statistical Error +/- 0.027646906817950512
epoch 5
Average Training MSE: 0.007554243221984991
Average Testing MSE: 0.006873914834924964
Statistical Error +/- 0.0262181517939861
epoch 6
Average Training MSE: 0.0068363486294275785
Average Testing MSE: 0.0063593481586721136
Statistical Error +/- 0.02521774803322476
epoch 7
Average Training MSE: 0.006204078547350259
Av

KeyboardInterrupt: 

In [ ]:
from datetime import datetime
import os

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
os.makedirs(f"./ckpts/{timestamp}")
for i, model in enumerate(models):
    torch.save(model.state_dict(), f"./ckpts/{timestamp}/model_{i}.pt")

In [ ]:
atwts = pd.read_csv("./data_constants/atomicweight.txt", sep="\t", header=None)
atwts.columns = ["atno", "sym", "atwt"]
atwts.set_index("sym", inplace=True)
atwts.atwt

elements = ["mg", "al", "si", "ca", "ti", "fe", "o"]

ele_wts = atwts.loc[elements].atwt

oxides = ["mgo", "al2o3", "sio2", "cao", "tio2", "feo"]

owts = {
    "mgo": ele_wts["o"] / (ele_wts["mg"] + ele_wts["o"]),
    "al2o3": ele_wts["o"] * 3 / (2 * ele_wts["al"] + 3 * ele_wts["o"]),
    "sio2": ele_wts["o"] * 2 / (ele_wts["si"] + 2 * ele_wts["o"]),
    "cao": ele_wts["o"] / (ele_wts["ca"] + ele_wts["o"]),
    "tio2": ele_wts["o"] * 2 / (ele_wts["ti"] + 2 * ele_wts["o"]),
    "feo": ele_wts["o"] / (ele_wts["fe"] + ele_wts["o"]),
}

orelwt = list(owts.values())
orelwt = torch.tensor(orelwt).to("mps")

NameError: name 'pd' is not defined

In [ ]:
def inference(input):
    _ = [model.eval() for model in models]
    outputs = torch.stack([model(input)[1] for model in models])
    print(outputs.shape)
    oxides = outputs.cpu().detach().numpy().mean(axis=0)
    ox_contrib = oxides * orelwt
    elems = oxides - ox_contrib
    oxygen = oxides.sum(axis=1)

    return np.append(elems, oxygen[:, np.newaxis], axis=1)

torch.Size([1024, 2])
torch.Size([10, 1024, 6])
(1024, 7)
